In [1]:
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType 

/home/nuri/dev/sources/git/iterative_ai_test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("json", data_files= "./data/sentiment_train_data/datasets.json")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1777.25it/s]
Generating train split: 5000 examples [00:00, 70186.04 examples/s]


In [4]:
dataset['train'][0]

{'sentence': '캠핑 다니며 그리들 사용해보니 모든게 맛있고 너무 좋더군요.  그래서 이번에 그리들팬 새로 장만해봤습니다^^  기존에 33cm 쓰다가 바꿀 때 되어서 구입했어요. 36, 40중에서 고민하다 40 샀는데 생각보다많이 크네요. 아무래도 캠핑에 가지고 다니기엔 부피가 너무 크네요. 그래도 가격도 좋고 깊이감도 있어 좋아요. 캠핑은 아직 못 가서 집에서 라면도 먹고 볶음밥도 먹고 고기도 먹었는데 맛있게 잘돼요ㅎ 구성품에 가방까지 있어서 좋아요.  상자가 다 찢어지고 배송은 완전 엉망이었어요.ㅠㅠ 다행히 상품은 이상 없었지만, 처음 찢어진 상자를 봤을땐 많이 놀랬네요 ㅋㅋ  여기까지 만족스러운 그리들 구매 후기를 남겨봤습니다~',
 'id': 1035734,
 'label': 0.0}

In [6]:
dataset.keys()

dict_keys(['train'])

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(dataset["train"])

,sentence,id,label
0,주변사람 추천템으로 인기가 많은 상품을 지금부터 소개합니다. 늘 사용하고 있는 제품입니다. 변기에 있는 물때를 깨끗하게 제거해주니 정말 좋은 것 같아요. 또한 오염방지 기능도 있어서 참 좋습니다. 방향제 효과도 있어서 사용할 때 향이 은은하게 나니 더 좋은 것 같아요. 다른 향보다 레몬향이 은은하게 오래 지속되는 점이 참 좋은 것 같네요. 가격이 그렇게 저렴한 편은 아니지만 다 쓰고 나면 재구매할 의사가 있습니다. 너무 좋아요. 내 삶을 더욱 업그레이드해주는 잇템이 될것 같습니다.,1035953,1.0
1,아기때부터 지금 어른이 된 이 순간까지 꾸준히 제 곁을 지켜준 이가 있는데요. 궁금하시다면 지금부터 후기속으로 빠져보겠습니다. 향이 은은하게 퍼져서 마음까지 편안하게 만들어주는 것 같습니다. 세안뿐만 아니라 바디용으로도 사용하기 좋은 것 같습니다. 세안을 했을 때 당기지 않고 촉촉해서 참 만족스러워요. 유통기한도 2년이나 남아서 엄청 넉넉합니다. 잘 구매한 것 같네요. 오늘은 여기까지 입니다 다소 부족하지만 이해해 주세요 다음에는 다른 내용으로 다시올게요!,1037003,1.0
2,40대가 되었음에도 ㅇㅇ는 영원한 사랑이네요 오늘은 ㅇㅇ 휴지를 소개해 볼까 합니다 ㅇㅇ를 너무 좋아해서 한 번 주문해봤다가 먼지도 안 나고 부드러워서 몇 년째 사용중입니다일단 디자인이 ㅇㅇ라 사용하면서도 행복하네요 롤도 단단히 감겨 있고 먼지도 거의 안나서 너무 좋습니다 재질도 고급지고 부드러움은 또 말할 수 없이 부드러워요 두께도 사용하기 적당합니다 너무 두꺼워도 불편할 때가 있거든요 가격도 ㅇㅇ지만 저렴하고 행사하면 더 좋은 가격에 구매할 수 있어요 강추합니다 포스팅을 보시고 더 좋은 선택을 할수 있는 의미있는 글이였으면 좋겠네요,1033869,1.0
3,오래된 개인주택이라 여기저기 생기는 곰팡이 때문에 고민이 많았는데 지인이 소개해 준 곰팡이 제거제가 있어 소개해드리려구요. 벽지에 결로땜에 곰팡이가 생겨서 뜯어내보니 안쪽에 곰팡이가 가득했어요. 너무 여러군데 곰팡이가 많아서 뿌리고 15분 정도 둔거 같아요. 그리고 젖은 걸레로 닦아냈는데 어머나~!!! 깜쪽같이... 진짜 깨끗하게 닦였어요. 대신 그만큼 냄새도 독한거 같아서 환기를 오랫동안 했지요. 사용할 때 마스크도 잘끼고 했구요...근데 뿌려만 두고 물로 씻어낸다고 해서 깨끗이 닦이지는 않아요. 솔 등으로 문질러주면 효과가 더 좋습니다. 곰팡이 제거 뿐만아니라 효과도 오래 지속되는 제품이라 좋은 정보 드리고 싶었어요. 다음에 또 다른 제품 포스팅 올릴께요.,1038029,1.0
4,여러분은 지금 어떤 생각을 하고 있습니까? 혹시 ㅇㅇㅇ 치약의 후기를 기다리고 계시진 않았나요? 지금 시작하겠습니다. 잇몸때문에 치과를 다니다가 추천을 받아서 구매를 했습니다. 이시림이 효과가 있을까 생각을 했는데 확실히 좋아졌네요. 이를 닦고 나면 입안이 상쾌해서 좋습니다. 개운함이 오래유지되는 점도 좋은 것 같고요. 가격이 비싼편인데 온라인으로 구매해도 비싼 것 같습니다. 만족스러운 치약을 만난 것 같습니다. 저는 이만 물러갈께요! 부족한 글이지만 끝까지 읽어주셔서 너무 고마워요~ 다음에도 많이 준비 할테니 또 오세요,1036956,1.0
5,말복만 지나면 이제 코스모스 피고 아침저녁으로 서늘한 바람이 불 것입니다. 그 생각으로 기운을 차려 오늘을 보내야 하겠습니다. 전에 사용하던 먼지털이개가 너무 오래되고 잘 안 닦여서 새상품으로 구입을 했습니다 화면 그대로 디자인이 예뻐서 맘에 듭니다 차량의 먼지도 시원하게 잘 털려서 아주 좋습니다 극세사라서 차량에 흠집이 안나서 마음에 들어요 손잡이 길이도 늘려져서 사용하기 좋습니다 가격도 저렴해서 딱 좋네요 오늘 내용이 좋은 정보가 되였나요? 그랬다면 좋아요와 댓글 부탁드려요 다음에도 더 좋은 내용으로 찾아뵐께요,1034626,1.0
6,며칠전 길을 걷다 앞에 걸어가는 사람에게서 나는 은은한 향이 꼭 이 향 같았어요 향도 은은하게 오래 남아서 매일 쓰기 좋아요 가끔 향이 강한 타사 제품 섬유유연제는 머리 가 아픈데 그렇지 않아 좋네요 고급스러운 은은한향이라 맘에 들고 시트형은 처음 사용해봤 는데 정말 편하고 한장씩 낱개포장이라 더 사용하기 편하고 좋아요. 반신반의하며 샀는데 비누향이 정말 좋네요 단점은 좀 비싸네요 여기에서 좋은 가격에 사긴했어요 비싼만큼 효과가 좋으니 만족합니다 늘 사용하는 섬유유연제지만 마음에 드는 제품 없으시다면 이 제품 한번 사용해보시라고 하 고 싶네요,1026944,1.0
7,"안녕하세요 이웃님들~~OOO 바르는 곰팡이싹 사용후기 지금 시작합니다. 욕실 벽 모퉁이에 곰팡이가 생겨 쭉 짜두고 한시간 방치하니 노랗게 곰팡이 부분이 분해되더니 젤도 노란 색으로 바뀌더라구요. 독한 냄새가 납니다. 호흡기엔 좋지 않으니 마스크 쓰고 하는 게 좋다고 봐요. 노랗게 변한 젤을 닦고 환기를 시켰어요. 확연히 없어진 곰팡이를 보니 속이 다 시원합니다. 묽지 않아 주르륵 흘러내리지 않아서 이렇게 애매한 스팟에도 사용 가능하니 좋아요. 정말 고난도의 스킬을 필요로 하지 않아 강추합니다.단, 오래동안 사용하지 않으면 좀 굳어요. 설명 끝!! 좀 도움이 되셨나요? 도움이 되셨다면 전 너무 행복해요~~",1030262,1.0
8,많은 분들에게 인기가 많은 ㅇㅇㅇ 화장지 후기를 지금부터 작성하려고 합니다 1+1 행사라 너무 좋네요 ㅇㅇㅇ 비싼 제품이라 늘 망설이는데 이번에 하나 가격으로 보니 너무 저렴한거 같아 바로 구매했습니다 3겹이라 두툼하고 타브랜드 3겹보다 더 두툼합니다 길이도 30m나 되니 좋고 향도 은은해서 더욱 마음에 듭니다 ㅇㅇㅇ 휴지는 먼지도 덜하네요 아예 없는건 아닌데 신경 쓰일 정도는 아닙니다 30롤 2팩해서 60롤이나 되니 오랫동안 잘 사용하겠어요 만족합니다 제 후기로 인해 구매시 많은 도움이 되었으면 좋겠습니다,1033795,1.0
9,오늘도 방문해 주셔서 감사합니다. 오늘의 후기는 밀폐용기입니다. 우리 밀폐용기도 이왕 예쁜 게 좋잖아요? 저는 디자인이 이쁘면 저도 모르게 사는 경향이 있 는데요. 이 제품 뚜껑 색이 파스텔 톤으로 너무 예뻐서 색은 정말 마음에 들었는데요. 뚜껑 소재가 좀 저렴한 플라스틱 느낌이어서 오래 쓸 수 있을지는 의문이었어요. 그리고 전자레인지에 사용 가능하다고 해서 돌렸는데 찌그러졌어요. 전자레인지에 오래 돌릴 수 있는 소재는 아닌 것 같아요. 올 한해 잘 마무리하시고 항상 건강하고 행복하세요.,1032748,-1.0


In [9]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [10]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Generating test split: 100%|██████████| 1063/1063 [00:00<00:00, 79723.65 examples/s]
/tmp/ipykernel_42102/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [12]:
dataset["train"][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

In [13]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [15]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


In [18]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [19]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [20]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 1063/1063 [00:00<00:00, 71780.04 examples/s]


In [21]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [24]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.